In [ ]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

parser = argparse.ArgumentParser()
parser.add_argument("--epochs", type=int, default=30, help="number of epochs")
parser.add_argument("--image_folder", type=str, default="data/samples", help="path to dataset")
parser.add_argument("--batch_size", type=int, default=16, help="size of each image batch")
parser.add_argument("--model_config_path", type=str, default="config/yolov3.cfg", help="path to model config file")
parser.add_argument("--data_config_path", type=str, default="config/coco.data", help="path to data config file")
parser.add_argument("--weights_path", type=str, default="weights/yolov3.weights", help="path to weights file")
parser.add_argument("--class_path", type=str, default="data/coco.names", help="path to class label file")
parser.add_argument("--conf_thres", type=float, default=0.8, help="object confidence threshold")
parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation")
parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
parser.add_argument(
    "--checkpoint_dir", type=str, default="checkpoints", help="directory where model checkpoints are saved"
)
parser.add_argument("--use_cuda", type=bool, default=True, help="whether to use cuda if available")
opt = parser.parse_args(args=['--model_config_path', '/home/dllab/final/yolov3.cfg', '--data_config_path', '/home/dllab/final/clothes.data', '--class_path', '/home/dllab/final/clothes.names', '--checkpoint_interval', '10', '--checkpoint_dir', '/tmp/work', '--epoch', '121', '--batch_size', '7', '--weights_path', '/tmp/work/105.pt'])
print(opt)

cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

classes = load_classes(opt.class_path)

# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = data_config["train"]

# Get hyper parameters
hyperparams = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

# Initiate model
model = Darknet(opt.model_config_path)
# model.load_weights(opt.weights_path)
model = torch.load(opt.weights_path)        #use this to load
#model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train()

# Get dataloader
dataloader = torch.utils.data.DataLoader(
    ListDataset(train_path), batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu
)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

for epoch in range(opt.epochs):
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        
        imgs = imgs.to('cuda') 
        targets = targets.to('cuda') 

        optimizer.zero_grad()

        loss = model(imgs, targets)

        loss.backward()
        optimizer.step()

        print(
            "[Epoch %d/%d, Batch %d/%d] [Losses: x %f, y %f, w %f, h %f, conf %f, cls %f, total %f, recall: %.5f, precision: %.5f]"
            % (
                epoch,
                opt.epochs,
                batch_i,
                len(dataloader),
                model.losses["x"],
                model.losses["y"],
                model.losses["w"],
                model.losses["h"],
                model.losses["conf"],
                model.losses["cls"],
                loss.item(),
                model.losses["recall"],
                model.losses["precision"],
            )
        )

        model.seen += imgs.size(0)

    if epoch % opt.checkpoint_interval == 0:
        #model.save_weights("%s/%d.weights" % (opt.checkpoint_dir, epoch))
        torch.save(model, "%s/%d.pt" % (opt.checkpoint_dir, epoch))


Namespace(batch_size=7, checkpoint_dir='/tmp/work', checkpoint_interval=10, class_path='/home/dllab/final/clothes.names', conf_thres=0.8, data_config_path='/home/dllab/final/clothes.data', epochs=121, image_folder='data/samples', img_size=416, model_config_path='/home/dllab/final/yolov3.cfg', n_cpu=0, nms_thres=0.4, use_cuda=True, weights_path='/tmp/work/105.pt')
[Epoch 0/121, Batch 0/143] [Losses: x 0.014281, y 0.015409, w 0.024643, h 0.023405, conf 0.048614, cls 0.672135, total 0.798488, recall: 0.56863, precision: 0.09254]
[Epoch 0/121, Batch 1/143] [Losses: x 0.018559, y 0.019965, w 0.031780, h 0.022340, conf 0.070540, cls 0.633701, total 0.796885, recall: 0.62500, precision: 0.10704]
[Epoch 0/121, Batch 2/143] [Losses: x 0.014751, y 0.013127, w 0.031286, h 0.027017, conf 0.048986, cls 0.621724, total 0.756892, recall: 0.64286, precision: 0.08332]
[Epoch 0/121, Batch 3/143] [Losses: x 0.021709, y 0.025441, w 0.030546, h 0.020780, conf 0.060347, cls 0.622029, total 0.780852, recall:

[Epoch 0/121, Batch 47/143] [Losses: x 0.030118, y 0.019067, w 0.049393, h 0.037155, conf 0.062198, cls 0.658855, total 0.856785, recall: 0.66667, precision: 0.11164]
[Epoch 0/121, Batch 48/143] [Losses: x 0.030651, y 0.026126, w 0.030972, h 0.022142, conf 0.087156, cls 0.640494, total 0.837542, recall: 0.69048, precision: 0.05778]
[Epoch 0/121, Batch 49/143] [Losses: x 0.028313, y 0.026125, w 0.029839, h 0.026666, conf 0.074636, cls 0.627170, total 0.812749, recall: 0.50877, precision: 0.06467]
[Epoch 0/121, Batch 50/143] [Losses: x 0.043153, y 0.017618, w 0.027892, h 0.020950, conf 0.068079, cls 0.612771, total 0.790463, recall: 0.68889, precision: 0.07916]
[Epoch 0/121, Batch 51/143] [Losses: x 0.031128, y 0.018071, w 0.040535, h 0.032863, conf 0.094213, cls 0.598829, total 0.815640, recall: 0.66667, precision: 0.06881]
[Epoch 0/121, Batch 52/143] [Losses: x 0.023762, y 0.024436, w 0.048345, h 0.037653, conf 0.092927, cls 0.639268, total 0.866391, recall: 0.59649, precision: 0.06863

[Epoch 0/121, Batch 97/143] [Losses: x 0.041444, y 0.054758, w 0.042675, h 0.056230, conf 0.286426, cls 0.621729, total 1.103261, recall: 0.74603, precision: 0.06752]
[Epoch 0/121, Batch 98/143] [Losses: x 0.038033, y 0.034024, w 0.078454, h 0.076916, conf 0.131717, cls 0.646031, total 1.005174, recall: 0.61111, precision: 0.04084]
[Epoch 0/121, Batch 99/143] [Losses: x 0.040324, y 0.045399, w 0.051493, h 0.055233, conf 0.143938, cls 0.628254, total 0.964642, recall: 0.56410, precision: 0.02970]
[Epoch 0/121, Batch 100/143] [Losses: x 0.032607, y 0.046301, w 0.025297, h 0.029801, conf 0.168439, cls 0.638666, total 0.941111, recall: 0.49206, precision: 0.04077]
[Epoch 0/121, Batch 101/143] [Losses: x 0.044885, y 0.044856, w 0.077526, h 0.076738, conf 0.172174, cls 0.649656, total 1.065836, recall: 0.60494, precision: 0.05595]
[Epoch 0/121, Batch 102/143] [Losses: x 0.093089, y 0.075363, w 0.085669, h 0.025402, conf 0.186187, cls 0.680332, total 1.146043, recall: 0.56667, precision: 0.03

[Epoch 1/121, Batch 3/143] [Losses: x 0.051740, y 0.060387, w 0.059585, h 0.057849, conf 0.154362, cls 0.644094, total 1.028016, recall: 0.65333, precision: 0.05400]
[Epoch 1/121, Batch 4/143] [Losses: x 0.063017, y 0.042574, w 0.073742, h 0.056017, conf 0.308381, cls 0.640639, total 1.184370, recall: 0.56410, precision: 0.05208]
[Epoch 1/121, Batch 5/143] [Losses: x 0.056347, y 0.130140, w 0.052375, h 0.078749, conf 0.151760, cls 0.622933, total 1.092304, recall: 0.64912, precision: 0.05311]
[Epoch 1/121, Batch 6/143] [Losses: x 0.073266, y 0.061417, w 0.048525, h 0.046866, conf 0.134518, cls 0.650725, total 1.015317, recall: 0.61905, precision: 0.04853]
[Epoch 1/121, Batch 7/143] [Losses: x 0.056309, y 0.047255, w 0.047103, h 0.056750, conf 0.143034, cls 0.657266, total 1.007716, recall: 0.56410, precision: 0.03139]
[Epoch 1/121, Batch 8/143] [Losses: x 0.058009, y 0.089728, w 0.082148, h 0.065360, conf 0.163639, cls 0.649536, total 1.108420, recall: 0.63889, precision: 0.05578]
[Epo

[Epoch 1/121, Batch 53/143] [Losses: x 0.066640, y 0.052206, w 0.097015, h 0.062199, conf 0.131133, cls 0.662439, total 1.071633, recall: 0.54902, precision: 0.03146]
[Epoch 1/121, Batch 54/143] [Losses: x 0.050862, y 0.057422, w 0.068483, h 0.054679, conf 0.131635, cls 0.637399, total 1.000480, recall: 0.64815, precision: 0.05413]
[Epoch 1/121, Batch 55/143] [Losses: x 0.058952, y 0.028412, w 0.061784, h 0.060081, conf 0.123919, cls 0.635500, total 0.968647, recall: 0.64286, precision: 0.03729]
[Epoch 1/121, Batch 56/143] [Losses: x 0.050502, y 0.040286, w 0.066804, h 0.082520, conf 0.166472, cls 0.678441, total 1.085025, recall: 0.61458, precision: 0.06041]
[Epoch 1/121, Batch 57/143] [Losses: x 0.040538, y 0.036069, w 0.038979, h 0.042691, conf 0.142695, cls 0.616898, total 0.917871, recall: 0.69231, precision: 0.04264]
[Epoch 1/121, Batch 58/143] [Losses: x 0.053137, y 0.052026, w 0.046896, h 0.042323, conf 0.149980, cls 0.655251, total 0.999613, recall: 0.65517, precision: 0.06188

[Epoch 1/121, Batch 103/143] [Losses: x 0.026366, y 0.025470, w 0.039787, h 0.044242, conf 0.094506, cls 0.633277, total 0.863648, recall: 0.56250, precision: 0.04665]
[Epoch 1/121, Batch 104/143] [Losses: x 0.040507, y 0.033888, w 0.027150, h 0.038958, conf 0.096584, cls 0.598747, total 0.835835, recall: 0.61111, precision: 0.03759]
[Epoch 1/121, Batch 105/143] [Losses: x 0.029055, y 0.031878, w 0.038591, h 0.053095, conf 0.090888, cls 0.635024, total 0.878532, recall: 0.66667, precision: 0.04760]
[Epoch 1/121, Batch 106/143] [Losses: x 0.045998, y 0.045863, w 0.042586, h 0.040930, conf 0.108590, cls 0.630327, total 0.914293, recall: 0.61905, precision: 0.06310]
[Epoch 1/121, Batch 107/143] [Losses: x 0.027637, y 0.034319, w 0.037147, h 0.020157, conf 0.095138, cls 0.666471, total 0.880869, recall: 0.57407, precision: 0.05633]
[Epoch 1/121, Batch 108/143] [Losses: x 0.041427, y 0.039005, w 0.045387, h 0.037992, conf 0.175924, cls 0.696960, total 1.036694, recall: 0.46914, precision: 0

[Epoch 2/121, Batch 9/143] [Losses: x 0.025089, y 0.036816, w 0.029727, h 0.030927, conf 0.061237, cls 0.704395, total 0.888192, recall: 0.44444, precision: 0.04442]
[Epoch 2/121, Batch 10/143] [Losses: x 0.030962, y 0.033490, w 0.045827, h 0.029155, conf 0.092140, cls 0.642652, total 0.874227, recall: 0.69333, precision: 0.11157]
[Epoch 2/121, Batch 11/143] [Losses: x 0.021586, y 0.031178, w 0.037912, h 0.016933, conf 0.064399, cls 0.618458, total 0.790465, recall: 0.66667, precision: 0.07205]
[Epoch 2/121, Batch 12/143] [Losses: x 0.037082, y 0.024018, w 0.050798, h 0.042286, conf 0.064529, cls 0.620681, total 0.839396, recall: 0.65217, precision: 0.11984]
[Epoch 2/121, Batch 13/143] [Losses: x 0.030867, y 0.016800, w 0.030768, h 0.042193, conf 0.052775, cls 0.593998, total 0.767401, recall: 0.76923, precision: 0.08771]
[Epoch 2/121, Batch 14/143] [Losses: x 0.015802, y 0.037330, w 0.066724, h 0.040911, conf 0.067429, cls 0.632322, total 0.860518, recall: 0.64583, precision: 0.10888]

[Epoch 2/121, Batch 59/143] [Losses: x 0.016129, y 0.024452, w 0.046451, h 0.035040, conf 0.071073, cls 0.605493, total 0.798639, recall: 0.70833, precision: 0.07091]
[Epoch 2/121, Batch 60/143] [Losses: x 0.029147, y 0.032584, w 0.054813, h 0.036881, conf 0.095439, cls 0.619334, total 0.868199, recall: 0.65079, precision: 0.06391]
[Epoch 2/121, Batch 61/143] [Losses: x 0.022749, y 0.024450, w 0.057992, h 0.067135, conf 0.078278, cls 0.607642, total 0.858246, recall: 0.76471, precision: 0.08114]
